In [3]:
import tfrecord as tfr
import h5py
import os,sys
import numpy as np
import glob
import pandas as pd
from tqdm import tqdm
import shutil
def _parse_description(csv_path):
        label_type = pd.read_csv(csv_path, usecols=["label","type"])
        description = {}
        for _, row in label_type.iterrows():
            description[str(row['label']).strip()] = str(row['type']).strip()
        return description
tfr_description = _parse_description("label_type.csv")
save_dir = "raw_data_removed"
os.makedirs(save_dir, exist_ok=True)
for path in glob.glob("raw_data/*.tfrecord"):
    loader = tfr.tfrecord_loader(path, None, tfr_description  )
    for record in loader:
        # print(record.keys())
        if record['MPI3_fixed']>0.00001:
            shutil.copy(path, os.path.join(save_dir, os.path.basename(path)))
        
print('ok')


ok


In [4]:
import tfrecord as tfr
import h5py
import os,sys
import numpy as np
import glob
import pandas as pd
from tqdm import tqdm
class TfrecordWorker():
    def __init__(self,tfr_list):
        
        self.info = {"label":[],"typee":[],"shape":[]}
        self.data_dir = "raw_data_removed"
        self.tfr_list = tfr_list
        self.tfr_description = self._parse_description("label_type.csv")
        loader = tfr.tfrecord_loader(self.tfr_list[0], None, self.tfr_description  )
        for record in loader:
            for key in record.keys():
                self.info['label'].append(key)
                self.info['typee'].append(type(record[key][0]))
                self.info['shape'].append(record[key].shape)
        self.attr_size = len(self.info['label'])
        self.data_size = len(self.tfr_list)
        print(f"总共有{self.attr_size}个属性")
        print(f"总共有{self.data_size}个tfrecord文件")


    def create_h5f(self, h5path="./data.h5"):
        self.h5f = h5py.File(h5path, 'w')
        self.dset = {}
        for i in range(self.attr_size):
            label = self.info["label"][i]
            typee = self.info["typee"][i]
            shape = self.info["shape"][i]
            if shape[0]==255*255:
                self.dset[label] = self.h5f.create_dataset(label,
                                shape=[self.data_size, 255,255],
                                compression=None,
                                dtype=typee)
            else:
                self.dset[label] = self.h5f.create_dataset(label,
                                shape=[self.data_size, shape[0]],
                                compression=None,
                                dtype=typee)
                                
        # self.dset["country"] = self.h5f.create_dataset("country",
        #                     shape=[self.data_size],
        #                     compression=None,
        #                     dtype=h5py.special_dtype(vlen=str))
        
        self.dset["name"] = self.h5f.create_dataset("name",
                            shape=[self.data_size],
                            compression=None,
                            dtype=h5py.special_dtype(vlen=str))
    def write_h5f(self):
        for idx,tfr_path in tqdm(enumerate(self.tfr_list)):
            self._write_one_item(tfr_path, idx)
            # if idx>5:
            #     break
            
        
    def close_h5f(self):
        self.h5f.close()

    def _write_one_item(self, tfr_path, idx):
        loader = tfr.tfrecord_loader(tfr_path, None, self.tfr_description  )
        for record in loader:
            # print(record['country'])
            for key in record.keys(): 
                content = record[key]
                if content.shape[0]==255*255:
                    content = np.reshape(content, (255,255))
                self.dset[key][idx] = content
        self.dset["name"][idx] = tfr_path.split("/")[-1]

    def _parse_description(self, csv_path):
        label_type = pd.read_csv(csv_path, usecols=["label","type"])
        description = {}
        for _, row in label_type.iterrows():
            description[str(row['label']).strip()] = str(row['type']).strip()
        return description



def start(files, savename):
    worker = TfrecordWorker(files)
    worker.create_h5f(savename)
    worker.write_h5f()
    worker.close_h5f()

# start(glob.glob("raw_data/*.tfrecord"),"data.h5")
start(glob.glob("raw_data_removed/*fold0*.tfrecord"),"subsetfold0.h5")
start(glob.glob("raw_data_removed/*fold1*.tfrecord"),"subsetfold1.h5")
start(glob.glob("raw_data_removed/*fold2*.tfrecord"),"subsetfold2.h5")
start(glob.glob("raw_data_removed/*fold3*.tfrecord"),"subsetfold3.h5")
# start(glob.glob("raw_data/*fold0*.tfrecord"),"test.h5")

总共有56个属性
总共有434个tfrecord文件


434it [02:10,  3.33it/s]


总共有56个属性
总共有362个tfrecord文件


362it [01:54,  3.17it/s]


总共有56个属性
总共有350个tfrecord文件


350it [02:08,  2.73it/s]


总共有56个属性
总共有334个tfrecord文件


334it [02:05,  2.66it/s]


In [1]:
import h5py
f = h5py.File('fold0.h5', 'r')
print('--iterms: ', len(f.keys()), f.keys())
for k in f.keys():
    print(k, f[k].shape)

--iterms:  57 <KeysViewHDF5 ['A', 'AOD047', 'AOD055', 'BLUE', 'DHSCLUST1', 'EVI', 'GREEN', 'H', 'Households_num', 'Individual_num', 'LAT', 'LON', 'Living Standards', 'MPI3', 'MPI3_fixed', 'MPI_Easy4', 'MPI_Easy4_fixed', 'NDVI', 'NIGHTLIGHTS', 'NIR', 'Number of Households2', 'RED', 'SWIR1', 'SWIR2', 'TEMP1', 'age_struct_child', 'age_struct_middle', 'age_struct_old', 'age_struct_young', 'area', 'aspect', 'building_area', 'burnedCount', 'conflict_num', 'country', 'death_num', 'education', 'elevation', 'famale', 'health', 'lat', 'lon', 'male', 'name', 'osm', 'place_num', 'poi_num', 'pr', 'pr_sum', 'road_length', 'slope', 'tmm_sum', 'tmmn', 'tmmx', 'viirs_v2', 'water', 'year']>
A (448, 1)
AOD047 (448, 255, 255)
AOD055 (448, 255, 255)
BLUE (448, 255, 255)
DHSCLUST1 (448, 1)
EVI (448, 255, 255)
GREEN (448, 255, 255)
H (448, 1)
Households_num (448, 1)
Individual_num (448, 1)
LAT (448, 255, 255)
LON (448, 255, 255)
Living Standards (448, 1)
MPI3 (448, 1)
MPI3_fixed (448, 1)
MPI_Easy4 (448, 1)
M